## INTRODUCTION

This project is targeted to build a database for storing data related to crimes that occured in Boston. 

### CREATING THE CRIME DATABASE

In [37]:
import psycopg2
conn = psycopg2.connect(dbname ="dq", user="dq")
cur = conn.cursor()
conn.autocommit = True
cur.execute("CREATE DATABASE crime_db")
conn.close()

ProgrammingError: database "crime_db" already exists


The database already exists, so we can disconnect the dq database name and be connected with the crime_db

In [ ]:
conn = psycopg2.connect(dbname='crime_db', user='dq')
cur = conn.cursor()
conn.autocommit = True
cur.execute("CREATE SCHEMA crimes")

### OBTAINING COLUMN NAMES AND SAMPLE FROM DATASET

In [38]:
import csv
with open('boston.csv', 'r') as file:
    reader = csv.reader(file)
    col_headers = next(reader)
    first_row = next(reader)
#   

### Creating a function for analyzing column values

In [39]:
def get_col_set(csv_filename, col_index):
    import csv
    values = set()
    with open(csv_filename) as f:
        next(f)
        reader = csv.reader(f)
        for row in reader:
            values.add(row[col_index])
    return values

for i in range(len(col_headers)):
    values = get_col_set('boston.csv', i)
    print(col_headers[i], len(values), sep='\t')

incident_number	298329
offense_code	219
description	239
date	1177
day_of_the_week	7
lat	18177
long	18177


### ANALYZING THE MAXIMUM LENGTH OF THE DESCRIPTIONS COLUMN

In [40]:
print(col_headers)

['incident_number', 'offense_code', 'description', 'date', 'day_of_the_week', 'lat', 'long']


In [41]:
descriptions = get_col_set('boston.csv', 2)
maximum_length = 0
for description in descriptions:
    maximum_length = max(maximum_length, len(description))
print(maximum_length)

58


In [42]:
print(col_headers)
print(first_row)

['incident_number', 'offense_code', 'description', 'date', 'day_of_the_week', 'lat', 'long']
['1', '619', 'LARCENY ALL OTHERS', '2018-09-02', 'Sunday', '42.35779134', '-71.13937053']


In [43]:
cur.execute("""
    CREATE TYPE weekday_enum AS ENUM('Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday')
""")
cur.execute("""
    CREATE TABLE crimes.boston_crimes(
        incident_number INTEGER PRIMARY KEY,
        offence_code INTEGER,
        descriptions VARCHAR(100),
        date DATE,
        day_of_the_week weekday_enum,
        lat DECIMAL,
        long DECIMAL
    )
""")


ProgrammingError: type "weekday_enum" already exists


In [ ]:
import csv
with open('boston.csv', 'r') as file:
    cur.copy_expert("COPY crimes.boston_crimes FROM STDIN WITH CSV HEADER", file)
    cur.execute("SELECT * FROM crimes.boston_crimes")
    print(len(cur.fetchall()))

In [44]:
cur.execute("REVOKE ALL ON SCHEMA public FROM PUBLIC")
cur.execute("REVOKE ALL ON DATABASE crime_db from public")

### Creating GROUP (Readonly)

In [45]:
cur.execute("CREATE GROUP readonly NOLOGIN")
cur.execute("GRANT USAGE ON SCHEMA crimes.boston_crimes TO readonly")
cur.execute("GRANT SELECT ON ALL TABLES IN SCHEMA crimes TO readonly")
cur.execute("GRANT CONNECT ON DATABASE crime_db TO readonly")

ProgrammingError: role "readonly" already exists


### Creating  GROUP (readwrite)

We create a `readwrite` group with `NOLOGIN` because it is a group and not a user. We grant the group the ability to connect to the `crime_db` and the ability to use the `crimes` schema.

Then we deal wit tables privileges by granting `SELECT`, `INSERT`, `UPDATE` and `DELETE`. As before we change the default privileges so that user in the `readwrite` group have these privileges if we ever create a new table on the `crimes` schema.

In [46]:
cur.execute("CREATE GROUP readwrite NOLOGIN")
cur.execute("GRANT USAGE ON SCHEMA crimes.boston_crimes TO readwrite")
cur.execute("GRANT SELECT, INSERT, UPDATE, DELETE ON ALL SCHEMA crimes TO readwrite")
cur.execute("GRANT CONNECT ON DATABASE crime_db TO readwrite")

ProgrammingError: role "readwrite" already exists


In [49]:
cur.execute("CREATE USER data_analyst WITH PASSWORD 'secret1'")
cur.execute("GRANT readonly TO data_analyst")

ProgrammingError: role "data_analyst" already exists


In [50]:
cur.execute("CREATE USER data_scientist WITH password 'secret2'")
cur.execute("GRANT readwrite TO data_scientist")

ProgrammingError: role "data_scientist" already exists


In [52]:
cur.execute("""
    SELECT grantee, privilege_type
    FROM information_schema.table_privileges
    WHERE grantee = 'readwrite'
""")

In [54]:
conn.close()
conn = psycopg2.connect(dbname='crime_db', user='dq')
cur = conn.cursor()
cur.execute("""
    SELECT rolname, rolsuper, rolcreaterole, rolcreatedb, rolcanlogin
    FROM pg_roles
    WHERE rolname IN ('readonly', 'readwrite', 'data_analyst', 'data_scientist')
""")
for user in cur:
    print(user)
print()
cur.execute("""
    SELECT grantee, privilege_type
    FROM information_schema.table_privileges
    WHERE grantee IN ('readonly', 'readwrite')
""")
for user in cur:
    print(user)
conn.close()

('readonly', False, False, False, False)
('readwrite', False, False, False, False)
('data_scientist', False, False, False, True)
('data_analyst', False, False, False, True)

